In [1]:
import gurobipy as grb
from gurobipy import GRB
import scipy.sparse as spr
import numpy as np
import random
import matplotlib.pyplot as plt
#from sympy import symbols, Rational
from IPython.display import display, Math, Markdown
import numpy.ma as ma


In [758]:
%run /Users/enzo-macmini/auction_alg_ITU/parallel_auction_ITU.ipynb

In [759]:
n_small = 4
m_small = 3
example_small = OneToOneITU( n = n_small,m = m_small, size_params= (4,24) , random_seed= 778, lbs =(0,0), tol = 1e-1)

In [869]:
def iteration(self,i_ent, mu, V_j):
    C  = np.zeros(self.n, dtype= bool)
    C_i_ent = np.zeros(self.n, dtype= bool)
    C[i_ent] = 1
    B = np.ones(self.m, dtype= bool)
    L_j = np.ones(self.m, dtype= int) *(self.n + 1)
    d_j = np.ones(self.m + 1, dtype= int) * np.inf

    i_j_diffs = np.zeros([self.n, self.m])

    iter = 0
    while np.any(C == 1):
        iter += 1
        i = np.argmax(C)
        print(f"iter {iter}, i: {i}")
        C[i] = 0
        C_i_ent[i] = 1
        U_ij_s = self.get_U_ij(V_j, i)[0]

        π_i = np.max(U_ij_s)
        
        suboptimal_js = U_ij_s < π_i 
      
        A_ij_star_max = np.max(self.A_ij[i,~suboptimal_js ])

        i_j_diffs[i, suboptimal_js] =  (π_i - U_ij_s[suboptimal_js] ) /A_ij_star_max


        d_j[:-1][suboptimal_js] = np.minimum(d_j[:-1][suboptimal_js], (π_i - U_ij_s[suboptimal_js] ) /A_ij_star_max) 
        d_j[-1] = np.minimum(  d_j[-1],   (π_i - self.lb_U) / A_ij_star_max)
      
        unissigned_optimal_js = np.where((mu.sum(0) == 0) * (π_i == U_ij_s))[0]

        if len(unissigned_optimal_js)> 0 :

            i_t = i
            j_t = unissigned_optimal_js[0]
      
            while i_t != i_ent :
 
                j_t_plus_1 = np.where(mu[i_t,: ]>0)[0][0]
          
                mu[i_t,: ] = 0
                mu[i_t, j_t] = 1
           
                w_i_s = np.sort( np.concatenate([self.get_U_ij(V_j, i_t)[0], [self.lb_U]]))[-2]
         
                V_j[j_t] = self.get_V_ij(i_t  , j_t,  w_i_s)

                j_t = j_t_plus_1
                i_t = L_j[j_t]
           

            w_i_s = np.sort( np.concatenate([self.get_U_ij(V_j, i_t)[0], [self.lb_U]]))[-2]
            mu[i_t,: ] = 0
            mu[i_t, j_t] = 1

            V_j[j_t] = self.get_V_ij(i_t  , j_t,  w_i_s)
            # print(f"j : {j}")
            #print(L_j)
            print("augmentation")
            return mu, V_j
            
        
        for j in np.where(B * (~suboptimal_js) >0)[0]:
            
            #U_ij = self.get_U_ij(V_j, i, j)[0]
            if mu[:,j].sum() == 1:
                i_j = np.argmax(mu[:,j])
                B[j] = 0
                C[i_j] = 1
                
                L_j[j] = i if L_j[j] == self.n + 1 else None
                #print(f"j : {j}")
                #print(L_j)
                print(f"i:{i} -> j:{j} ")


    matrix = i_j_diffs[ :, B * (d_j[:-1] < np.inf)]  
    masked_matrix = np.ma.masked_where(matrix <= 0, matrix)
    argmin_pos = np.unravel_index(np.argmin(masked_matrix), masked_matrix.shape)


    # 
    i_low = argmin_pos[0]
    U_ij_s = self.get_U_ij(V_j, i_low)[0]
    π_i = np.max(U_ij_s)
    optimal_js = U_ij_s == π_i 
    j_low = np.argmax(self.A_ij[i_low,optimal_js])


    return C_i_ent, B * (d_j[:-1] < np.inf), d_j

OneToOneITU.iteration = iteration

In [860]:
mu_ex = np.zeros([n_small,m_small])
V_ex = np.ones(example_small.m) * example_small.lb_V
perm_unassigned = np.zeros(example_small.n, dtype= bool)

In [861]:
example_small.A_ij 

array([[1.33333333, 6.        , 0.33333333, 0.88888889, 2.        ],
       [1.25      , 0.375     , 7.        , 0.75      , 0.25      ],
       [0.33333333, 0.57142857, 2.33333333, 1.33333333, 0.33333333],
       [2.33333333, 4.5       , 0.44444444, 0.6       , 3.        ],
       [1.5       , 1.        , 0.5       , 0.66666667, 1.8       ]])

In [862]:
example_small.B_ij

array([[8, 1, 7, 0, 3],
       [8, 3, 8, 0, 3],
       [2, 9, 2, 1, 9],
       [9, 9, 4, 3, 7],
       [7, 9, 0, 6, 4]])

In [871]:
#C_i = np.zeros(example_small.n, dtype= bool)

for p in range(1):

    i_ent = np.where((mu_ex.sum(1) == 0) * (~perm_unassigned) > 0)[0][0]
    iter_values = example_small.iteration(i_ent, mu_ex, V_ex )

    if len(iter_values) == 2:
        mu_ex, V_ex = iter_values

    else:
        C_i, B, d = iter_values
        print(B)
        B_with_out = np.concatenate( (B, [True]) )

        V_ex[~B] += np.min(d[B_with_out])

        print(d)
        print(B)
        mu_ex[C_i,:] = 0
        busy = np.zeros(example_small.m, dtype= bool)
        for i in np.where(C_i > 0)[0]:
            j_i = np.argmax( example_small.get_U_ij(V_ex, i) )
            #print(j_i)
            if busy[j_i] == False:
                mu_ex[:,j_i] = 0
                mu_ex[i,j_i] = 1
                busy[j_i] = 1


       

        #print(B)
        print(f"d:{np.min(d[B_with_out])}")
        print(f"iteration total:{p}")
    
        

    perm_unassigned = np.all(example_small.get_U_ij(V_ex , np.arange(example_small.n)) <= example_small.lb_U + 1e-14, axis = 1)
mu_ex[perm_unassigned,:] = 0

iter 1, i: 3
i:3 -> j:0 
i:3 -> j:1 
iter 2, i: 1
iter 3, i: 4
jjjj
[2.33333333 4.5       ]
[False False  True  True  True]
[2.79948802 3.31213861 0.10953444 0.9109623  0.02207341 1.57762897]
[False False  True  True  True]
d:0.02207341269841301
iteration total:0


In [711]:
example_small.A_ij

array([[0.19047619, 0.85714286, 0.04761905, 0.12698413, 0.28571429],
       [0.17857143, 0.05357143, 1.        , 0.10714286, 0.03571429],
       [0.04761905, 0.08163265, 0.33333333, 0.19047619, 0.04761905],
       [0.33333333, 0.64285714, 0.06349206, 0.08571429, 0.42857143],
       [0.21428571, 0.14285714, 0.07142857, 0.0952381 , 0.25714286]])

In [712]:
mu_ex

array([[0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.]])

In [713]:
example_small.get_U_ij(V_ex , np.arange(example_small.n)). round(3)

array([[ 6.354, -2.841,  6.927,  0.   ,  2.413],
       [ 6.457,  2.76 ,  6.457,  0.   ,  2.927],
       [ 1.588,  8.634,  1.486,  1.   ,  8.902],
       [ 6.119,  6.119,  3.902,  3.   ,  6.119],
       [ 5.148,  8.36 , -0.11 ,  6.   ,  3.472]])

In [704]:
example_small.get_U_ij(V_ex , np.arange(example_small.n))[3,0] - example_small.get_U_ij(V_ex , np.arange(example_small.n))[3,1] 

9.089557826413852e-07

In [801]:
import numpy as np

matrix = np.array([[1, 0, 0], [0, -5, 6], [7, 8, 9]])

argmin_pos = np.unravel_index(np.argmin(matrix[matrix > 0]), matrix.shape)
print(f"Argmin of positive entries: {argmin_pos}")


Argmin of positive entries: (0, 0)


In [161]:
def ITU_auction(self):
    mu_ij = np.zeros([self.n,self.m])
    V_j = np.ones(self.m) * self.lb_V
    perm_unassigned = np.zeros(self.n, dtype= bool)
    iters = 0
    for p in range(10000):
        iters += 1
        i_ent = np.where((mu_ij.sum(1) == 0) * (~perm_unassigned) > 0)[0]

        if len(i_ent)> 0:
            i_ent = i_ent[0]

        else:
            id_i , id_j  =  np.where(mu_ij == 1)
    
            U_i = np.ones(self.n) * self.lb_U
            U_i[id_i] = self.get_U_ij(V_j, id_i)[np.arange(len(id_i)), id_j]  

            # print(iters) 
            return  mu_ij,U_i, V_j, perm_unassigned * 1, iters



        iter_values = self.iteration(i_ent, mu_ij, V_j )

        if len(iter_values) == 2:
            mu_ij, V_j = iter_values

        else:
            C_i, B, d = iter_values
            B_with_out = np.concatenate( (B, [True]) )

            V_j[~B] += np.min(d[B_with_out])

            # print(d)
            # print(B)
            mu_ij[C_i,:] = 0
            busy = np.zeros(self.m, dtype= bool)
            for i in np.where(C_i > 0)[0]:
                j_i = np.argmax( self.get_U_ij(V_j, i) )
                #print(j_i)
                if busy[j_i] == False:
                    mu_ij[:,j_i] = 0
                    mu_ij[i,j_i] = 1
                    busy[j_i] = 1


        

            #print(B)
            # print(f"d:{np.min(d[B_with_out])}")
            # print(f"iteration total:{p}")
        
            

        perm_unassigned = np.all(self.get_U_ij(V_j , np.arange(self.n)) <= self.lb_U+ 1e-14, axis = 1)
        mu_ij[perm_unassigned,:] = 0

    print(f"FAILURE, MAX ITERS {iters}")


    # id_i , id_j  =  np.where(mu_ij == 1)
    
    # U_i = np.ones(self.n) * self.lb_U
    # U_i[id_i] = self.get_U_ij(V_j, id_i)[np.arange(len(id_i)), id_j]   

    # return mu_ij,U_i, V_j, perm_unassigned * 1, iters

OneToOneITU.ITU_auction = ITU_auction

In [767]:
n_small = 5
m_small = 5
example_small = OneToOneITU( n = n_small,m = m_small, size_params= (10,10) , random_seed= 3066, lbs =(0,0), tol = 0)

In [186]:

# for i in range(1,100000):
#     n_small = 4
#     m_small = 4
#     example_small = OneToOneITU( n = n_small,m = m_small, size_params= (10,10) , random_seed= i, lbs =(0,0), tol = 0)
#     print(f"ITER{i}")
#     mu_ex , U_ex,  V_ex , perm_ , iters = example_small.ITU_auction()
#     if iters > 8000 :
#         print(f"ITER{i} AAAAAAA")
#         break


In [187]:
mu_ex , U_ex,  V_ex , perm_ , iters= example_small.ITU_auction()

FAILURE, MAX ITERS 10000


TypeError: cannot unpack non-iterable NoneType object

In [180]:
mu_ex

array([[0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.]])

In [181]:
example_small.check_all((mu_ex , U_ex , V_ex ))

____
<h1>Feasibility</h1>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

#matched: 4 over 4

____
<h1>Generalized Complementary Slackness</h1>

...

____
<h1>Individual Rationality</h1>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [2939]:
example_small.get_U_ij(V_ex , np.arange(10)) 

array([[ 4.        ,  2.        ,  1.        ,  4.        ,  4.        ,
         4.        ,  4.        ,  0.        ,  2.66071429,  0.        ],
       [ 2.        ,  4.        ,  0.        ,  4.        ,  3.        ,
         1.        ,  0.        ,  3.        ,  0.69093407,  2.        ],
       [ 1.        ,  3.        ,  0.        ,  0.        ,  1.        ,
         0.        ,  2.        ,  0.        ,  3.        ,  3.        ],
       [ 1.        ,  1.        ,  2.        ,  3.        ,  1.        ,
         4.        ,  2.        ,  4.        ,  3.58791209,  1.        ],
       [ 0.        ,  1.        ,  2.        ,  4.        ,  2.        ,
         0.        ,  1.        ,  1.        ,  3.82142857,  1.        ],
       [ 2.        ,  2.        ,  4.        ,  4.        ,  0.        ,
         2.        ,  4.        ,  2.        ,  2.5511583 ,  3.        ],
       [ 4.        ,  1.        ,  4.        ,  3.        ,  1.        ,
         2.        ,  1.        ,  0.        

In [2503]:
example_small. check_dual_feas(U_ex,V_ex, output = True)

<IPython.core.display.Math object>

(array([[-13.8351983 ,   0.        , -10.92857143,   0.        ,
          -3.28571429,   0.        ,   0.        ,   0.        ,
         -13.93650794,   0.        ],
        [  0.        ,   0.        ,   0.        ,   0.        ,
           0.        ,   0.        ,   0.        ,   0.        ,
           0.        ,   0.        ],
        [  0.        ,   0.        ,   0.        ,   0.        ,
           0.        ,   0.        ,   0.        ,   0.        ,
           0.        ,   0.        ],
        [  0.        ,   0.        ,   0.        ,   0.        ,
           0.        ,   0.        ,   0.        ,   0.        ,
           0.        ,   0.        ],
        [  0.        ,   0.        ,   0.        ,   0.        ,
           0.        ,   0.        ,   0.        ,   0.        ,
           0.        ,   0.        ],
        [  0.        ,   0.        ,   0.        ,   0.        ,
           0.        ,   0.        ,   0.        ,   0.        ,
           0.        ,   0.   

In [2038]:
mu_ex[0,:].sum()

1.0